# EXTRACCIÓN DE DATOS:

Paletas de sombras, pintalabios y colorete en función de la paleta de colores asignadas, siendo estas:\
        -PRIMAVERA\
        -VERANO\
        -OTOÑO\
        -INVIERNO

**IMPORTACIONES**

In [537]:
from selenium import webdriver
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
from joblib import Parallel, delayed
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re

In [ ]:
pd.set_option('display.max_rows', None)

Ahora, vamos a scrappear los productos de maquillaje con mejor rating que se ajusten a las paletas de colores\
de cada estación. Sacaremos nombre, precio y rating.

In [335]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)  

2022-12-12 16:24:52,097 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/235325829.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



*Existen varios productos que podemos recomendar segun la colometría de las personas. En este caso, nos centraremos\
en sombras de ojos, pintalabios, colorete y corrector*\

Por otra parte, debemos tener en cuenta que hay varias tiendas donde podemos encontrar productos de maquillaje.\
Concretamente, en nuestro proyecto, los escogeremos de las siguientes:

1. **PRIMOR**
2. **SEPHORA**

# **1 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS, PINTALABIOS Y COLORETE DE PRIMOR**

In [336]:
URL = 'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20verano&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and'
driver.get(URL)

In [4]:
#Investigamos dónde podemos entrar para scrappear PRIMOR
filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')

In [368]:
precios = driver.find_elements(By.CLASS_NAME, 'df-card__price')

In [370]:
#Paletas de sombras
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    total.drop(['TIPO'], axis = 1, inplace = True)
    
    driver.quit()
     
    return total

In [371]:
links_sombras_primor = ['https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20verano&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20oto%C3%B1al&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20invierno&filter%5Bcategories%5D%5B0%5D=Ojos%20lujo&filter%5Bcategories%5D%5B1%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B2%5D=Sombras%20de%20ojos%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20pastel&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B1%5D=Sombras%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [372]:
productos = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_sombras_primor)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   41.4s remaining:   41.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   43.0s finished


In [373]:
sombras_ver_prim = productos[0] 
sombras_oto_prim = productos[1] 
sombras_inv_prim = productos[2] 
sombras_pri_prim = productos[3] 
#productos_totales = pd.concat([productos[0], productos[1], productos[2], productos[3]], axis = 0)

In [374]:
ad1 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20krash%20rosas&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B1%5D=Sombras%20de%20ojos&filter%5Bbrand%5D%5B0%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad2 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20lilas&filter%5Bcategories%5D%5B0%5D=Maquillaje%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad3 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20negras&filter%5Bcategories%5D%5B0%5D=Maquillaje%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad4 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20naranjas&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')

2022-12-12 16:38:39,760 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

2022-12-12 16:39:00,221 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

2022-12-12 16:39:17,233 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

2022-12-12 16:39:41,491 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2921855445.py:3: 

In [375]:
#Limpiamos los índices y añadimos lo que hemos sacado más

#PRIMAVERA
sombras_pri_prim = pd.concat([sombras_pri_prim, ad1], axis = 0)
sombras_pri_prim.drop_duplicates(inplace = True)
sombras_pri_prim['ID'] = [e for e in range(len(sombras_pri_prim))]
sombras_pri_prim.set_index('ID', drop = True, inplace = True)

#OTOÑO
sombras_oto_prim = pd.concat([sombras_oto_prim, ad2], axis = 0)
sombras_oto_prim.drop_duplicates(inplace = True)
sombras_oto_prim['ID'] = [e for e in range(len(sombras_oto_prim))]
sombras_oto_prim.set_index('ID', drop = True, inplace = True)

#INVIERNO
sombras_inv_prim = pd.concat([sombras_inv_prim, ad3], axis = 0)
sombras_inv_prim.drop_duplicates(inplace = True)
sombras_inv_prim['ID'] = [e for e in range(len(sombras_inv_prim))]
sombras_inv_prim.set_index('ID', drop = True, inplace = True)

#VERANO
sombras_ver_prim = pd.concat([sombras_ver_prim, ad4], axis = 0)
sombras_ver_prim.drop_duplicates(inplace = True)
sombras_ver_prim['ID'] = [e for e in range(len(sombras_ver_prim))]
sombras_ver_prim.set_index('ID', drop = True, inplace = True)

In [409]:
sombras_pri_prim['PALETA'] = 'PRIMAVERA'
sombras_oto_prim['PALETA'] = 'OTOÑO'
sombras_inv_prim['PALETA'] = 'INVIERNO'
sombras_ver_prim['PALETA'] = 'VERANO'

In [11]:
#Pintalabios
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'PINTALABIOS'
    
    driver.quit()
     
    return total

In [12]:
links_pintalabios_primor = ['https://www.primor.eu/#/dfclassic/query=pintalabios%20vivid%20&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20neutro&filter%5Bcategories%5D%5B0%5D=Barras%20de%20labios%20lujo&filter%5Bcategories%5D%5B1%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20oscuro&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20rosados&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&filter%5Bbrand%5D%5B0%5D=Revolution&filter%5Bbrand%5D%5B1%5D=NYX%20Professional%20Makeup&filter%5Bbrand%5D%5B2%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [13]:
productos1 = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_pintalabios_primor)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   51.8s remaining:   51.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   52.7s finished


In [15]:
#Añadimos más filas
add1 = extraer('https://www.primor.eu/#/dfclassic/query=pintalabios%20rosados&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&filter%5Bbrand%5D%5B0%5D=NYX%20Professional%20Makeup&filter%5Bbrand%5D%5B1%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
add2 = extraer('https://www.primor.eu/#/dfclassic/query=berry%20nars&filter%5Bbrand%5D%5B0%5D=NARS&filter%5Bcategories%5D%5B0%5D=Labios%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_or')
add3 = extraer('https://www.primor.eu/#/dfclassic/query=nude%20nars&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
add4 = extraer('https://www.primor.eu/#/dfclassic/query=dark%20nars&filter%5Bbrand%5D%5B0%5D=NARS&filter%5Bcategories%5D%5B0%5D=Barras%20de%20labios%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_or')
add5 = extraer('https://www.primor.eu/#/dfclassic/query=mac%20lips&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')

2022-12-12 10:57:07,258 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

2022-12-12 10:57:27,066 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

2022-12-12 10:57:48,343 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

2022-12-12 10:58:03,451 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2764708321.py:3: 

In [16]:
pintalab_ver_prim = productos1[0] 
pintalab_oto_prim = productos1[1] 
pintalab_inv_prim = productos1[2] 
pintalab_pri_pri = productos1[3]

In [17]:
#Limpiamos los índices y añadimos lo que hemos sacado más

#PRIMAVERA
pintalab_pri_prim = pd.concat([pintalab_pri_pri, add1, add3], axis = 0)
pintalab_pri_prim.drop_duplicates(inplace = True)
pintalab_pri_prim['ID'] = [e for e in range(len(pintalab_pri_prim))]
pintalab_pri_prim.set_index('ID', drop = True, inplace = True)

#OTOÑO
pintalab_oto_prim = pd.concat([pintalab_pri_pri, add1], axis = 0)
pintalab_oto_prim.drop_duplicates(inplace = True)
pintalab_oto_prim['ID'] = [e for e in range(len(pintalab_oto_prim))]
pintalab_oto_prim.set_index('ID', drop = True, inplace = True)

#INVIERNO
pintalab_inv_prim = pd.concat([pintalab_pri_pri, add4], axis = 0)
pintalab_inv_prim.drop_duplicates(inplace = True)
pintalab_inv_prim['ID'] = [e for e in range(len(pintalab_inv_prim))]
pintalab_inv_prim.set_index('ID', drop = True, inplace = True)

#VERANO
pintalab_ver_prim = pd.concat([pintalab_pri_pri, add5], axis = 0)
pintalab_ver_prim.drop_duplicates(inplace = True)
pintalab_ver_prim['ID'] = [e for e in range(len(pintalab_ver_prim))]
pintalab_ver_prim.set_index('ID', drop = True, inplace = True)

In [410]:
pintalab_pri_prim['PALETA'] = 'PRIMAVERA'
pintalab_oto_prim['PALETA'] = 'OTOÑO'
pintalab_inv_prim['PALETA'] = 'INVIERNO'
pintalab_ver_prim['PALETA'] = 'VERANO'

In [422]:
#Colorete
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'COLORETE'
    
    driver.quit()
     
    return total

In [19]:
links_blush_primor = ['https://www.primor.eu/3ina/66421-colorete-the-blush.html#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=MAC&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=NARS&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush%20rosa&filter%5Bbrand%5D%5B0%5D=3ina&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=Yves%20Saint%20Laurent&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [20]:
productos2 = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_blush_primor)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2688360952.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2688360952.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2688360952.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2688360952.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   35.4s remaining:   35.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   43.1s finished


In [21]:
blush_pri_prim = productos1[0] 
blush_ver_prim = productos1[1] 
blush_inv_prim = productos1[2] 
blush_oto_prim = productos1[3]

In [437]:
blush_pri_prim['PALETA'] = 'PRIMAVERA'
blush_ver_prim['PALETA'] = 'VERANO'
blush_inv_prim['PALETA'] = 'INVIERNO'
blush_oto_prim['PALETA'] = 'OTOÑO'

In [438]:
maquillaje_primor = pd.concat([sombras_pri_prim, sombras_ver_prim, sombras_oto_prim, sombras_inv_prim, \
                              pintalab_pri_prim, pintalab_ver_prim, pintalab_oto_prim, pintalab_inv_prim, \
                              blush_pri_prim, blush_ver_prim, blush_oto_prim, blush_inv_prim])

In [439]:
maquillaje_primor.drop('TIPO', axis = 1, inplace = True)

In [440]:
def limpiar_precios(columna):
    return columna.split()[0]

In [452]:
p = []
for e in maquillaje_primor.PRECIO:
    if ',' in e:
        p.append(e.replace(',', '.'))
        
        
maquillaje_primor.PRECIO = p

In [454]:
maquillaje_primor.rename(columns = {'PRECIO': 'PRECIO (€)'}, inplace = True ) 

In [441]:
maquillaje_primor.PRECIO = maquillaje_primor.PRECIO.apply(limpiar_precios)

In [442]:
maquillaje_primor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 29
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NOMBRE    543 non-null    object
 1   PRECIO    543 non-null    object
 2   LINKS     543 non-null    object
 3   PRODUCTO  543 non-null    object
 4   PALETA    543 non-null    object
dtypes: object(5)
memory usage: 25.5+ KB


In [443]:
#Añadimos la columna 'TIENDA' para indicar dónde comprar los productos
maquillaje_primor['TIENDA'] = 'PRIMOR'

In [463]:
#Cambiamos el tipo de dato a nuestro nuevo dataframe
maquillaje_primor[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA', 'TIENDA']] = maquillaje_primor[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA', 'TIENDA']].astype('category')
maquillaje_primor['PRECIO (€)'] = maquillaje_primor['PRECIO (€)'].astype('float')

In [464]:
maquillaje_primor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 29
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   NOMBRE      543 non-null    category
 1   PRECIO (€)  543 non-null    float64 
 2   LINKS       543 non-null    category
 3   PRODUCTO    543 non-null    category
 4   PALETA      543 non-null    category
 5   TIENDA      543 non-null    category
dtypes: category(5), float64(1)
memory usage: 28.8 KB


# **2 - EXTRACCIÓN DE DATOS DE SOMBRAS Y COLORETES DE URBAN DECAY**

In [22]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)  

2022-12-12 10:59:17,915 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/235325829.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [23]:
URL = 'https://www.urbandecay.com/eyeshadow/?prefn1=taxonomyColorEye&prefv1=BLACK%7CBLUE%7CBOLD%7CBROWN%7CCOOL%7CNEUTRALS%7CPLUMS%7CPURPLE%7CRED%7CSIENNA%7CSILVER%7CSMOKEY%7CTAUPE&start=0&sz=12#view-mode=grid'
driver.get(URL)

In [24]:
#Investigamos dónde podemos scrappear los datos que necesitamos
prueba = driver.find_elements(By.CLASS_NAME, 'c-product-tile__caption-inner')
nombres = driver.find_element(By.CLASS_NAME, 'c-product-tile__name').text
links1 = driver.find_elements(By.CLASS_NAME, 'c-product-image__link')
links = [e.get_attribute('href') for e in links1]
precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price') 
#precios = [e.text.split()[-1] for e in precio]


IndexError: list index out of range

In [25]:
#SOMBBRAS
def extraer(URL):
    PATH=ChromeDriverManager().install()
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, 'body>div.c-modal.h-text-align-center.m-borderless.m-autosize-large.m-image.m-opened.h-fade-in>div>button').click() 
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    nombre = driver.find_elements(By.CLASS_NAME, 'c-product-tile__name') #nombre
    time.sleep(5)
    precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price') #precio
    time.sleep(5)
    links1 = driver.find_elements(By.CLASS_NAME, 'c-product-image__link') #links
        
    tmp = [e.text for e in nombre]
    
    precios = [e.text.split('\n')[-1] for e in precio]
    
    links = [e.get_attribute('href') for e in links1]
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    driver.quit()
    
    return total

In [26]:
URL = 'https://www.urbandecay.com/naked-palettes/?start=0&sz=18'

In [27]:
sombras_ud = extraer(URL)

2022-12-12 11:00:04,873 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2965688531.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [28]:
sombras_ud.drop_duplicates(inplace = True)
sombras_ud = sombras_ud[0:14] 

In [29]:
#Añadimos los valores de las paletas en función de los colores de las sombras

sombras_ud['PALETA'] = ['PRIMAVERA', 'PRIMAVERA / OTOÑO', 'OTOÑO / INVIERNO', 'VERANO', 'TODAS', 'PRIMAVERA / VERANO',\
                  'OTOÑO', 'PRIMAVERA / VERANO', 'OTOÑO / INVIERNO', 'PRIMAVERA', 'PRIMAVERA / INVIERNO', 'PRIMAVERA',\
                  'PRIMAVERA / OTOÑO / INVIERNO', 'OTOÑO / INVIERNO']

In [30]:
#COLORETE
def extraer(URL):
    PATH=ChromeDriverManager().install()
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, 'body>div.c-modal.h-text-align-center.m-borderless.m-autosize-large.m-image.m-opened.h-fade-in>div>button').click() 
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    nombre = driver.find_elements(By.CLASS_NAME, 'c-product-tile__name') #nombre
    time.sleep(5)
    precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price') #precio
    time.sleep(5)
    links1 = driver.find_elements(By.CLASS_NAME, 'c-product-image__link') #links
        
    tmp = [e.text for e in nombre]
    
    precios = [e.text.split('\n')[-1] for e in precio]
    
    links = [e.get_attribute('href') for e in links1]
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'COLORETE'
    
    driver.quit()
    
    return total

In [31]:
blush_ud = extraer('https://www.urbandecay.com/blush-bronzer-highlighter/')

2022-12-12 11:00:48,393 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/169910402.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [32]:
blush_ud 

,NOMBRE,PRECIO,LINKS,PRODUCTO
0,"STAY NAKED THREESOME BRONZE, BLUSH, AND HIGHLI...",$37.00,https://www.urbandecay.com/threesome-by-urban-...,COLORETE
1,BEACHED BRONZER,$31.00,https://www.urbandecay.com/beached-bronzer-by-...,COLORETE
2,NAKED3 EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-3-palette-by-...,COLORETE
3,NAKED RELOADED EYESHADOW PALETTE,$22.00,https://www.urbandecay.com/naked-reloaded-urba...,COLORETE
4,NAKED HEAT EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-heat-eyeshado...,COLORETE
5,24/7 GLIDE-ON WATERPROOF EYELINER PENCIL,$11.50,https://www.urbandecay.com/24-7-glide-on-eye-p...,COLORETE
6,,,https://www.urbandecay.com/all-nighter-setting...,COLORETE
7,,,https://www.urbandecay.com/naked-honey-eyeshad...,COLORETE
8,,,https://www.urbandecay.com/eyeshadow-primer-po...,COLORETE
9,,,https://www.urbandecay.com/urban-decay-cosmeti...,COLORETE


In [33]:
blush_ud = blush_ud[0:2]
blush_ud.drop_duplicates(inplace = True)
blush_ud['PALETA'] = ['VERANO / INVIERNO', 'TODAS']


2022-12-12 11:01:22,134 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3659531634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blush_ud.drop_duplicates(inplace = True)

2022-12-12 11:01:22,165 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3659531634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blush_ud['PALETA'] = ['VERANO / INVIERNO', 'TODAS']



In [501]:
mu_ud = pd.concat([sombras_ud, blush_ud], axis = 0)

In [502]:
mu_ud

,NOMBRE,PRECIO,LINKS,PRODUCTO,PALETA
0,NAKED3 EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-3-palette-by-...,SOMBRAS DE OJOS,PRIMAVERA
1,NAKED RELOADED EYESHADOW PALETTE,$22.00,https://www.urbandecay.com/naked-reloaded-urba...,SOMBRAS DE OJOS,PRIMAVERA / OTOÑO
2,NAKED HEAT EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-heat-eyeshado...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
3,NAKED HONEY EYESHADOW PALETTE,$24.50,https://www.urbandecay.com/naked-honey-eyeshad...,SOMBRAS DE OJOS,VERANO
4,LET'S GET NAKED SET,$175.00,https://www.urbandecay.com/lets-get-naked-set-...,SOMBRAS DE OJOS,TODAS
5,UD X ROBIN EISENBERG ARTIST COLLAB HOLIDAY MAK...,$49.00,https://www.urbandecay.com/robin-eisenberg-hol...,SOMBRAS DE OJOS,PRIMAVERA / VERANO
6,NAKED WILD WEST MINI EYESHADOW PALETTE,$18.90,https://www.urbandecay.com/naked-wild-west-min...,SOMBRAS DE OJOS,OTOÑO
7,NAKED X ROBIN EISENBERG EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/robin-eisenberg-nak...,SOMBRAS DE OJOS,PRIMAVERA / VERANO
8,NAKED WILD WEST EYESHADOW PALETTE,$24.50,https://www.urbandecay.com/urban-decay-cosmeti...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
9,NAKED3 MINI EYESHADOW PALETTE,$29.00,https://www.urbandecay.com/naked3-mini-eyeshad...,SOMBRAS DE OJOS,PRIMAVERA


In [504]:
#Vamos a limpiar la columna de PRECIO
p2 = []
for e in mu_ud['PRECIO']:
    if '$' in e:
        p2.append(float(e.replace('$', '')))
        
#Lo convertimos a EUROS(TIPO DE CAMBIO - 12/12/2022):
p3 = []
for e in p2:
    p3.append(round((e*0.95), 2))

mu_ud.PRECIO = p3

In [513]:
mu_ud.rename(columns = {'PRECIO' : 'PRECIO (€)'}, inplace = True)

In [516]:
#Cambiamos el tipo de dato de las columnas 
mu_ud[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']] = mu_ud[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']].astype('category')

In [531]:
mu_ud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 1
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   NOMBRE      16 non-null     category
 1   PRECIO (€)  16 non-null     float64 
 2   LINKS       16 non-null     category
 3   PRODUCTO    16 non-null     category
 4   PALETA      16 non-null     category
dtypes: category(4), float64(1)
memory usage: 2.1 KB


# **3 - EXTRACCIÓN DE DATOS DE SOMBRAS Y COLORETES DE HUDA BEAUTY**

In [267]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH) 

2022-12-12 16:04:12,877 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/91254812.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [279]:
URL = 'https://hudabeauty.com/us/en_US/eyes/eyeshadow'
driver.get(URL)

In [319]:
#SOMBRAS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 4500);')  # 200px de scroll
    time.sleep(10)
    
    filas = driver.find_elements(By.CLASS_NAME, 'link')
    precios = driver.find_elements(By.CLASS_NAME, 'price')
    tmp = []
    links = []
    precio = []

    for e in filas:
        tmp.append(e.text)
    
    for e in filas:
        links.append(e.get_attribute('href'))
    
    for e in precios:
        precio.append(e.text.split()[0])
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRECIO'] = precio
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [320]:
URL = 'https://hudabeauty.com/us/en_US/eyes/eyeshadow'

In [321]:
sombras_hb = extraer(URL)

2022-12-12 16:19:59,725 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/958777798.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [525]:
sombras_hb['PALETA']=  ['PRIMAVERA / OTOÑO / INVIERNO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO','OTOÑO / INVIERNO',\
                        'PRIMAVERA / OTOÑO / INVIERNO', 'VERANO / OTOÑO', 'VERANO / OTOÑO / INVIERNO',\
                        'TODAS', 'VERANO', 'PRIMAVERA', 'PRIMAVERA / OTOÑO', 'PRIMAVERA', \
                        'OTOÑO / INVIERNO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA', 'OTOÑO / INVIERNO', 'PRIMAVERA / VERANO / OTOÑO',\
                        'OTOÑO / PRIMAVERA', 'TODAS', 'OTOÑO / VERANO', 'VERANO']

In [324]:
#LABIOS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 4500);')  # 200px de scroll
    time.sleep(10)
    
    filas = driver.find_elements(By.CLASS_NAME, 'link')
    precios = driver.find_elements(By.CLASS_NAME, 'price')
    tmp = []
    links = []
    precio = []

    for e in filas:
        tmp.append(e.text)
    
    for e in filas:
        links.append(e.get_attribute('href'))
    
    for e in precios:
        precio.append(e.text.split()[0])
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRECIO'] = precio
    total['PRODUCTO'] = 'PINTALABIOS'
    
    
    driver.quit()
     
    return total

In [325]:
URL = 'https://hudabeauty.com/us/en_US/makeup/lips'

In [326]:
pintalabios_hb = extraer(URL)

2022-12-12 16:21:28,932 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1998456133.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [524]:
paletas = ['TODAS', 'TODAS', 'TODAS', 'PRIMAVERA / VERANO / OTOÑO', 'TODAS', 'PRIMAVERA / OTOÑO / INVIERNO',\
          'TODAS', 'TODAS', 'TODAS', 'TODAS', 'INVIERNO', 'TODAS', 'TODAS', 'INVIERNO / OTOÑO', 'TODAS',\
          'OTOÑO / PRIMAVERA / INVIERNO', 'OTOÑO / INVIERNO', 'TODAS', 'TODAS', 'OTOÑO / INVIERNO']
pintalabios_hb['PALETA'] = paletas

In [329]:
#BLUSH
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 4500);')  # 200px de scroll
    time.sleep(10)
    
    filas = driver.find_elements(By.CLASS_NAME, 'link')
    precios = driver.find_elements(By.CLASS_NAME, 'price')
    tmp = []
    links = []
    precio = []

    for e in filas:
        tmp.append(e.text)
    
    for e in filas:
        links.append(e.get_attribute('href'))
    
    for e in precios:
        precio.append(e.text.split()[0])
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRECIO'] = precio
    total['PRODUCTO'] = 'COLORETE'
    
    
    driver.quit()
     
    return total

In [330]:
URL = 'https://hudabeauty.com/us/en_US/cheek/blush'

In [331]:
blush_hb = extraer(URL)

2022-12-12 16:22:34,520 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/4071659692.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [526]:
blush_hb['PALETA'] = 'TODAS'

In [527]:
maquillaje_hb = pd.concat([sombras_hb, pintalabios_hb, blush_hb])

In [528]:
#Vamos a limpiar la columna de PRECIO
p2 = []
for e in maquillaje_hb['PRECIO']:
    if '$' in e:
        p2.append(float(e.replace('$', '')))
#Lo convertimos a EUROS (TIPO DE CAMBIO - 12/12/2022):
p3 = []
for e in p2:
    p3.append(round((e*0.95), 2))

maquillaje_hb.PRECIO = p3

In [529]:
maquillaje_hb.rename(columns = {'PRECIO' : 'PRECIO (€)'}, inplace = True)

In [530]:
#Cambiamos el tipo de dato de las columnas 
maquillaje_hb[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']] = \
maquillaje_hb[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']].astype('category')

In [532]:
maquillaje_hb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 3
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   NOMBRE      45 non-null     category
 1   LINKS       45 non-null     category
 2   PRECIO (€)  45 non-null     float64 
 3   PRODUCTO    45 non-null     category
 4   PALETAS     45 non-null     object  
 5   PALETA      45 non-null     category
dtypes: category(4), float64(1), object(1)
memory usage: 4.7+ KB


# **4 - EXTRACCIÓN DE DATOS DE SOMBRAS,  COLORETES Y PINTALABIOS DE KIKO MILANO**

In [51]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH) 

2022-12-12 11:03:35,973 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/91254812.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [52]:
URL = 'https://www.kikocosmetics.com/es-es/maquillaje/labios.html'

In [53]:
driver.get(URL)

In [54]:
#Investigamos cómo scrappear la página de KIKO MILANO para poder sacar los productos y los precios
nombres = driver.find_elements(By.CLASS_NAME, 'ProductBox__Title')
precio = driver.find_elements(By.CLASS_NAME, 'js-new-price')
link2 =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Palette')

In [241]:
#BLUSH Y SOMBRAS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.find_element(By.XPATH, '//*[@id="iubenda-cs-banner"]/div/div/div/div[3]/div[2]/button[2]').click()
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    nombre = driver.find_elements(By.CLASS_NAME, 'ProductBox__Title')
    precio = driver.find_elements(By.CLASS_NAME, 'js-new-price')
    link =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Palette')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    precios.remove('')
    precios.remove('')
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [242]:
URL = 'https://www.kikocosmetics.com/es-es/maquillaje/paletas/paleta-de-ojos.html'

In [243]:
sombras_kiko = extraer(URL) #CUÁLES SOMBRAS Y CUÁLES BLUSH?

2022-12-12 13:45:54,438 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3280125201.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [244]:
sombras_kiko.shape

(28, 4)

In [567]:
paletas = ['PRIMAVERA / OTOÑO', 'TODAS', 'TODAS', 'TODAS', 'TODAS', 'TODAS', 'TODAS', 'TODAS', 'TODAS', 'TODAS','TODAS',\
          'TODAS', 'TODAS', 'PRIMAVERA / VERANO', 'PRIMAVERA / VERANO / OTOÑO', 'INVIERNO / OTOÑO', 'INVIERNO / OTOÑO', \
          'INVIERNO / OTOÑO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO / INVIERNO', 'TODAS', 'PRIMAVERA / OTOÑO / INVIERNO',\
          'PRIMAVERA / OTOÑO / INVIERNO', 'TODAS', 'PRIMAVERA / VERANO', 'PRIMAVERA / VERANO', 'TODAS', 'TODAS']
sombras_kiko['PALETA'] = paletas

In [568]:
sombras_kiko.at[26, 'PRODUCTO'] = 'COLORETE'
sombras_kiko.at[27, 'PRODUCTO'] = 'COLORETE'

In [569]:
#PINTALABIOS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    nombre = driver.find_elements(By.CLASS_NAME, 'ProductBox__Title')
    precio = driver.find_elements(By.CLASS_NAME, 'js-new-price')
    link =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Lip')
    link2 =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Unlimited')
    link3 =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Stick')
    link4 = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Pearly')
    link5 = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Jelly')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    for e in link2:
        links.append(e.get_attribute('href'))
    
    for e in link3:
        links.append(e.get_attribute('href'))
    
    for e in link4:
        links.append(e.get_attribute('href'))
    
    for e in link5:
        links.append(e.get_attribute('href'))
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'PINTALABIOS'
    
    
    driver.quit()
     
    return total

In [59]:
URL = 'https://www.kikocosmetics.com/es-es/maquillaje/labios.html'

In [60]:
labios_kiko = extraer(URL)

2022-12-12 11:04:04,420 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1661189169.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [570]:
labios_kiko['PALETA'] = 'TODAS'

In [571]:
maquillaje_kiko = pd.concat([sombras_kiko, labios_kiko])

In [572]:
#Limpiamos la columna PRECIO
p = []
for e in maquillaje_kiko.PRECIO:
    if ',' in e:
        p.append(e.replace(',', '.'))
        
        
maquillaje_kiko.PRECIO = p

#Limpiamos la columna PRECIO
def quitar_euro(columna):
    e = re.findall('\d+.\d+', columna)
    e = ''.join(e)
    return float(e)

maquillaje_kiko.PRECIO = maquillaje_kiko.PRECIO.apply(quitar_euro)

maquillaje_kiko.rename(columns = {'PRECIO' : 'PRECIO (€)'}, inplace = True)

In [573]:
#Cambiamos el tipo de dato de las columnas
maquillaje_kiko[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']] = \
maquillaje_kiko[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']].astype('category')

In [576]:
maquillaje_kiko.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 23
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   NOMBRE      52 non-null     category
 1   PRECIO (€)  52 non-null     float64 
 2   LINKS       52 non-null     category
 3   PRODUCTO    52 non-null     category
 4   PALETA      52 non-null     category
dtypes: category(4), float64(1)
memory usage: 6.3 KB


# 5 - EXTRACCIÓN DE DATOS DE PINTALABIOS DE KAT VON D

In [62]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

2022-12-12 11:04:22,654 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1981730666.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [63]:
URL = 'https://kvdveganbeauty.com/collections/lip'

In [64]:
driver.get(URL)

In [65]:
nombres = driver.find_elements(By.CLASS_NAME, 'product-card__name')
precios = driver.find_elements(By.CLASS_NAME, 'product-card__price')

In [66]:
lista = []
for e in nombres:
    lista.append(e.text)

In [67]:
lista[18:24]

['Everlasting Hyperlight Transfer-Proof Liquid Lipstick',
 'Epic Kiss Nourishing Vegan Butter Lipstick',
 'Mini Epic Kiss Nourishing Vegan Butter Lipstick',
 'Studded Kiss Crème Lipstick',
 'XO Vinyl Lip Cream Lip Gloss',
 'XO Lip Gloss']

In [68]:
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    time.sleep(5)
    driver.find_element(By.CLASS_NAME, 'glClose').click()
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    nombre = driver.find_elements(By.CLASS_NAME, 'product-card__name')
    precio = driver.find_elements(By.CLASS_NAME, 'product-card__price')
    link =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Lip')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    tmp = tmp[18:24]
    links = links[0:6]
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)

    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'PINTALABIOS'
    
    
    driver.quit()
     
    return total

In [69]:
URL = 'https://kvdveganbeauty.com/collections/lip'

In [70]:
pintalabios_kvd = extraer(URL) #falta añadir las paletas

2022-12-12 11:04:37,936 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3548246177.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [577]:
pintalabios_kvd['PALETA'] = 'TODAS'

In [578]:
pintalabios_kvd.drop('PALETAS', axis = 1, inplace = True)

In [580]:
#Limpiamos la columna PRECIO
def quitar_euro(columna):
    e = re.findall('\d+.\d+', columna)
    e = ''.join(e)
    return float(e)

pintalabios_kvd.PRECIO = pintalabios_kvd.PRECIO.apply(quitar_euro)
pintalabios_kvd.rename(columns = {'PRECIO' : 'PRECIO (€)'}, inplace = True)

TypeError: rename() got an unexpected keyword argument 'columns'

In [584]:
#Cambiamos el tipo de dato de las columnas
pintalabios_kvd[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']] = \
pintalabios_kvd[['NOMBRE', 'LINKS', 'PRODUCTO', 'PALETA']].astype('category')

# 5 - EXTRACCIÓN DE DATOS DE COLORETES DE BENEFIT

In [71]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

2022-12-12 11:05:13,513 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1981730666.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [72]:
URL = 'https://www.benefitcosmetics.com/en-us/face/blush?page=3'

In [73]:
driver.get(URL)

In [74]:
nombres = driver.find_elements(By.CLASS_NAME, 'card__title')
precios = driver.find_elements(By.CLASS_NAME, 'price')

In [75]:
link = driver.find_elements(By.TAG_NAME, 'a')

In [76]:
lista = []
for e in link:
    lista.append(e.get_attribute('href'))

In [77]:
lista[103:112]

['https://www.benefitcosmetics.com/en-us/product/moone-rich-berry-blush?product=MOONE-MINI',
 'https://www.benefitcosmetics.com/en-us/product/shellie-warm-seashell-pink-blush?product=SHELLIE-MINI',
 'https://www.benefitcosmetics.com/en-us/product/terra-golden-brick-red-blush?product=TERRA-MINI',
 'https://www.benefitcosmetics.com/en-us/product/peachin-golden-peach-blush?product=PEACHIN-MINI',
 'https://www.benefitcosmetics.com/en-us/product/dandelion-brightening-finishing-powder?product=DANDELION-FULL',
 'https://www.benefitcosmetics.com/en-us/product/georgia-golden-peach-blush?product=GEORGIA-FULL',
 'https://www.benefitcosmetics.com/en-us/product/georgia-golden-peach-blush?product=GEORFM233',
 'https://www.benefitcosmetics.com/en-us/product/love-me-bright',
 'https://www.benefitcosmetics.com/en-us/product/sincerely-yours-beauty-advent-calendar']

In [78]:
links = links[102:114]

In [79]:
lista[10:19]

['https://www.benefitcosmetics.com/en-us',
 'https://www.benefitcosmetics.com/en-us/locations',
 'https://www.benefitcosmetics.com/en-us/find-a-store',
 'https://www.benefitcosmetics.com/en-us/face/blush?page=3#',
 'https://www.benefitcosmetics.com/en-us/account/register/?returnUrl=%2Fen-us%2Fface%2Fblush',
 'https://www.benefitcosmetics.com/en-us/product/gimme-brow-volumizing-eyebrow-gel?product=GIMMEBROW-FULL',
 'https://www.benefitcosmetics.com/en-us/product/gimme-brow-volumizing-eyebrow-gel?product=GIMMEBROW-MINI',
 'https://www.benefitcosmetics.com/en-us/product/gimme-brow-volumizing-eyebrow-gel?product=GIMMEBROW-JUMBO',
 'https://www.benefitcosmetics.com/en-us/product/goof-proof-eyebrow-pencil?product=GOOFP-FULL']

In [80]:
precios = driver.find_elements(By.CLASS_NAME, 'price')

In [81]:
lista[102:114]

['https://www.benefitcosmetics.com/en-us/all-makeup',
 'https://www.benefitcosmetics.com/en-us/product/moone-rich-berry-blush?product=MOONE-MINI',
 'https://www.benefitcosmetics.com/en-us/product/shellie-warm-seashell-pink-blush?product=SHELLIE-MINI',
 'https://www.benefitcosmetics.com/en-us/product/terra-golden-brick-red-blush?product=TERRA-MINI',
 'https://www.benefitcosmetics.com/en-us/product/peachin-golden-peach-blush?product=PEACHIN-MINI',
 'https://www.benefitcosmetics.com/en-us/product/dandelion-brightening-finishing-powder?product=DANDELION-FULL',
 'https://www.benefitcosmetics.com/en-us/product/georgia-golden-peach-blush?product=GEORGIA-FULL',
 'https://www.benefitcosmetics.com/en-us/product/georgia-golden-peach-blush?product=GEORFM233',
 'https://www.benefitcosmetics.com/en-us/product/love-me-bright',
 'https://www.benefitcosmetics.com/en-us/product/sincerely-yours-beauty-advent-calendar',
 'https://www.benefitcosmetics.com/en-us/face/blush?page=3#',
 'https://www.benefitcos

In [82]:
driver.find_element(By.CSS_SELECTOR, '#category > div.category__container > div > div > div.pagination > a > i')

<selenium.webdriver.remote.webelement.WebElement (session="ad3fdbbc39be0cfd43f7e96d0f7c502b", element="32c913de-b5fa-4a91-a46d-3518d35e67e1")>

In [83]:
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)

    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    
    
    nombre = driver.find_elements(By.CLASS_NAME, 'card__title')
    precio = driver.find_elements(By.CLASS_NAME, 'price')
    link =  driver.find_elements(By.TAG_NAME, 'a')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text.split('\n')[-1])
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    precios = precios[10:22]
    tmp = tmp[10:22]
    links = links[102:114]
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)

    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'COLORETE'
    
    
    driver.quit()
     
    return total

In [84]:
def extraer2(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)

    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    
    
    nombre = driver.find_elements(By.CLASS_NAME, 'card__title')
    precio = driver.find_elements(By.CLASS_NAME, 'price')
    link =  driver.find_elements(By.TAG_NAME, 'a')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text.split('\n')[-1])
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    precios = precios[10:22]
    tmp = tmp[10:22]
    links = links[103:112]
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)

    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'COLORETE'
    
    
    driver.quit()
     
    return total

In [85]:
links_ben = ['https://www.benefitcosmetics.com/en-us/face/blush',\
            'https://www.benefitcosmetics.com/en-us/face/blush?page=2']

In [86]:
tres = extraer2('https://www.benefitcosmetics.com/en-us/face/blush?page=3') 

2022-12-12 11:05:31,868 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/326568913.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [87]:
blush_benefit = Parallel(n_jobs=3, verbose=True)(delayed(extraer)(e) for e in links_ben)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1349481635.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1349481635.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:   34.3s finished


In [88]:
blush1 = blush_benefit[0]
blush2 = blush_benefit[1]

In [218]:
blush1.drop(blush1.index[[6]], inplace = True)

In [221]:
paletas = ['PRIMAVERA / OTOÑO', 'TODAS', 'TODAS', 'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO', 'PRIMAVERA / OTOÑO',\
          'INVIERNO / VERANO', 'INVIERNO / VERANO', 'TODAS', 'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO']
blush1['PALETAS'] = paletas

In [223]:
paletas = ['PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO', 'INVIERNO / VERANO', 'INVIERNO / VERANO',\
          'INVIERNO / VERANO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO', 'PRIMAVERA / OTOÑO',\
          'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO']
blush2['PALETAS'] = paletas

In [225]:
paletas = ['INVIERNO / VERANO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO',\
          'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO', 'TODAS', 'TODAS']
tres['PALETAS'] = paletas

In [226]:
blush_benefit = pd.concat([blush1, blush2, tres], axis = 0)
blush_benefit['PRODUCTO'] = 'COLORETE'

In [228]:
blush_benefit

,NOMBRE,PRECIO,LINKS,PRODUCTO,PALETAS
0,Cheek the Mail,$36.00,https://www.benefitcosmetics.com/en-us/all-makeup,COLORETE,PRIMAVERA / OTOÑO
1,Cheek Party Package,$36.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,TODAS
2,Blush ‘n Brush Delivery,$30.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,TODAS
3,Forward to: Gorgeous,$50.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,PRIMAVERA / OTOÑO
4,Pretty Pink Postage,$20.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,INVIERNO / VERANO
5,Blushin’ Benetint Bundle,$20.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,PRIMAVERA / OTOÑO
7,Bright n' Peachy Blush Duo,$28.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,INVIERNO / VERANO
8,Whatta Beach!,$28.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,INVIERNO / VERANO
9,Wanderful World Blush Collection,$280.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,TODAS
10,Starlaa Rosy Bronze Blush,$29.00,https://www.benefitcosmetics.com/en-us/product...,COLORETE,PRIMAVERA / OTOÑO


# 5 - EXTRACCIÓN DE DATOS DE SOMBRAS, BLUSH Y PINTALABIOS DE NARS

In [90]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

2022-12-12 11:06:41,282 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1981730666.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [91]:
URL = 'https://www.narscosmetics.es/es/maquillaje/ojos/paletas-de-sombras'

In [92]:
driver.get(URL)

In [93]:
nombres = driver.find_elements(By.CLASS_NAME, 'product-name')

In [94]:
[e.text for e in nombres]

['SUMMER UNRATED EYESHADOW PALETTE',
 'UNWRAPPED MINI EYESHADOW PALETTE',
 'EUPHORIA FACE PALETTE',
 'COOL CRUSH EYESHADOW PALETTE',
 'VOYAGEUR EYESHADOW PALETTE',
 'VOYAGEUR EYESHADOW PALETTE',
 'VOYAGEUR EYESHADOW PALETTE',
 'VOYAGEUR EYESHADOW PALETTE',
 'VOYAGEUR EYESHADOW PALETTE',
 'CUARTETO DE SOMBRAS DE OJOS',
 'CUARTETO DE SOMBRAS DE OJOS',
 'CUARTETO DE SOMBRAS DE OJOS',
 'CUARTETO DE SOMBRAS DE OJOS',
 'CUARTETO DE SOMBRAS DE OJOS',
 '#22 BLENDING BRUSH',
 '#20 ALLOVER EYESHADOW BRUSH']

In [95]:
prec = driver.find_elements(By.CLASS_NAME, 'product-pricing')

In [96]:
[e.text.split()[0] for e in prec]

['55,00',
 '26,00',
 '55,00',
 '55,00',
 '30,50',
 '30,50',
 '30,50',
 '30,50',
 '30,50',
 '49,00',
 '49,00',
 '49,00',
 '49,00',
 '49,00',
 '31,50',
 '31,50']

In [97]:
driver.find_element(By.CLASS_NAME, 'name-link').get_attribute('href')

'/es/summer-unrated-eyeshadow-palette/0194251130200.html'

In [98]:
#SOMBRAS, LABIOS Y BRUSH
def extraer2(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)

    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, '#onetrust-accept-btn-handler').click()
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    nombre = driver.find_elements(By.CLASS_NAME, 'product-name')
    precio = driver.find_elements(By.CLASS_NAME, 'product-pricing')
    link =  driver.find_elements(By.CLASS_NAME, 'name-link')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text)
    
    for e in precio:
        precios.append(e.text.split()[0])
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)

    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [99]:
links_nars = ('https://www.narscosmetics.es/es/maquillaje/ojos/paletas-de-sombras',
             'https://www.narscosmetics.es/es/maquillaje/labios',
             'https://www.narscosmetics.es/es/maquillaje/rostro/blush')

In [100]:
nars = Parallel(n_jobs=3, verbose=True)(delayed(extraer2)(e) for e in links_nars)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2494103033.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2494103033.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/2494103033.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   56.1s finished


In [101]:
sombras_nars = nars[0]
labios_nars = nars[1]
blush_nars = nars[2]

In [204]:
sombras_nars
dictio = {'NOMBRE' : 'STARGAZE EYESHADOW PALETTE', 'PRECIO' : '59,00', 'LINKS' : 'https://www.narscosmetics.es/es/stargaze-eyeshadow-palette/0194251134321.html',\
         'PRODUCTO' : 'SOMBRAS DE OJOS'}
sombras_nars = sombras_nars.append(dictio, ignore_index = True)

2022-12-12 13:04:28,775 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/270403714.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sombras_nars = sombras_nars.append(dictio, ignore_index = True)



In [207]:
paletas = ['TODAS', 'PRIMAVERA / VERANO', 'PRIMAVERA / VERANO / OTOÑO', 'PRIMAVERA / OTOÑO', 'INVIERNO / OTOÑO',\
          'VERANO / OTOÑO', 'PRIMAVERA / VERANO', 'PRIMAVERA / OTOÑO', 'OTOÑO / INVIERNO', 'OTOÑO', 'PRIMAVERA / OTOÑO',\
          'PRIMAVERA', 'OTOÑO / VERANO', 'PRIMAVERA', 'PRIMAVERA / OTOÑO']
sombras_nars['PALETAS'] = paletas

In [210]:
paletas = ['VERANO / INVIERNO / OTOÑO', 'PRIMAVERA / VERANO', 'INVIERNO', 'PRIMAVERA', 'OTOÑO / INVIERNO',\
          'VERANO / INVIERNO / OTOÑO', 'PRIMAVERA', 'PRIMAVERA', 'PRIMAVERA / VERANO', 'TODAS', 'TODAS',\
          'VERANO / INVIERNO / OTOÑO', 'PRIMAVERA / VERANO', 'PRIMAVERA / VERANO', 'PRIMAVERA / VERANO', 'TODAS']
labios_nars['PALETAS'] = paletas

In [216]:
paletas = ['INVIERNO / VERANO', 'INVIERNO / VERANO', 'INVIERNO / VERANO', 'PRIMAVERA / OTOÑO', 'TODAS', \
          'PRIMAVERA / OTOÑO', 'TODAS', 'INVIERNO / VERANO', 'INVIERNO / VERANO', 'PRIMAVERA / OTOÑO', 'INVIERNO / VERANO',\
          'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO']
blush_nars['PALETAS'] = paletas

In [102]:
labios_nars['PRODUCTO'] = 'PINTALABIOS'
blush_nars['PRODUCTO'] = 'COLORETE'

In [103]:
def arreglar_links(link):
    return 'https://www.narscosmetics.es' + link 

sombras_nars.LINKS = sombras_nars.LINKS.apply(arreglar_links)

In [104]:
sombras_nars['LINKS'] = sombras_nars['LINKS'].apply(arreglar_links)
labios_nars['LINKS'] = labios_nars['LINKS'].apply(arreglar_links)
blush_nars['LINKS'] = blush_nars['LINKS'].apply(arreglar_links)

In [105]:
sombras_nars.drop(sombras_nars.index[[14,15]], inplace = True)
blush_nars.drop(blush_nars.index[[12,13,14]], inplace = True)

# 6 - EXTRACCIÓN DE DATOS DE SOMBRAS, BLUSH Y PINTALABIOS DE CHARLOTTE TILBURY

In [182]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

2022-12-12 12:39:52,662 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1981730666.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [183]:
URL = 'https://www.charlottetilbury.com/es/products/maquillaje/ojos/paleta'

In [184]:
driver.get(URL)

In [185]:
driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()

In [186]:
nombre = driver.find_element(By.CLASS_NAME, 'ProductCard__title').text.split('\n')[0]

In [187]:
nombre

'LUXURY PALETTE'

In [110]:
precio = driver.find_element(By.CLASS_NAME, 'DisplayPrice__price').text

In [111]:
links = driver.find_element(By.CLASS_NAME, 'ProductCard__link').get_attribute('href')

In [188]:
#SOMBRAS, LABIOS Y BLUSH
def extraer2(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()

    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    
    
    nombre = driver.find_elements(By.CLASS_NAME, 'ProductCard__title')
    precio = driver.find_elements(By.CLASS_NAME, 'DisplayPrice__price')
    link =  driver.find_elements(By.CLASS_NAME, 'ProductCard__link')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n')[0])
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)

    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [189]:
links_ct = ['https://www.charlottetilbury.com/es/products/maquillaje/ojos/paleta',\
           'https://www.charlottetilbury.com/es/products/maquillaje/labios/labial/revolucion-matte',\
           'https://www.charlottetilbury.com/es/products/maquillaje/mejillas/rubor/rubor-en-polvo']

In [190]:
ch_til = Parallel(n_jobs=3, verbose=True)(delayed(extraer2)(e) for e in links_ct)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3048332081.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3048332081.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/3048332081.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   32.0s finished


In [192]:
sombras_ct = ch_til[0]
labios_ct = ch_til[1]
blush_ct = ch_til[2]

In [196]:
paletas = ['OTOÑO', 'PRIMAVERA / OTOÑO / INVIERNO', 'INVIERNO', 'PRIMAVERA / IVNVIERNO', 'OTOÑO / INVIERNO',\
          'OTOÑO / INVIERNO']
sombras_ct['PALETAS'] = paletas

In [199]:
paletas = ['OTOÑO / INVIERNO', 'OTOÑO / INVIERNO / VERANO', 'PRIMAVERA / OTOÑO / VERANO', 'VERANO / OTOÑO, INVIERNO',\
          'OTOÑO / INVIERNO', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / VERANO', 'VERANO / INVIERNO', 'PRIMAVERA', 'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO']
labios_ct['PALETAS'] = paletas

In [200]:
paletas = ['VERANO / INVIERNO', 'PRIMAVERA / OTOÑO', 'VERANO / INVIERNO', 'VERANO / INVIERNO', 'PRIMAVERA / OTOÑO', \
          'VERANO / INVIERNO', 'VERANO / INVIERNO']
blush_ct['PALETAS'] = paletas

In [116]:
labios_ct['PRODUCTO'] = 'PINTALABIOS'
blush_ct['PRODUCTO'] = 'COLORETE'

# 7 - EXTRACCIÓN DE DATOS DE SOMBRAS, BLUSH Y PINTALABIOS DE BOBBI BROWN

In [118]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

2022-12-12 11:08:52,937 [48607] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/1981730666.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [119]:
URL = 'https://www.bobbibrown.es/products/2330/makeup/eyes/sombra-de-ojos'

In [120]:
driver.get(URL)

In [121]:
precio = driver.find_elements(By.CLASS_NAME, 'price')

In [122]:
def extraer2(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')
    time.sleep(5)
    
    
    nombre = driver.find_elements(By.CLASS_NAME, 'product-brief__header')
    precio = driver.find_elements(By.CLASS_NAME, 'price')
    link =  driver.find_elements(By.CLASS_NAME, 'product-brief__headline-review-link')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text)
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
        
    for e in precios:
        if e == '':
            precios.remove(e)
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [123]:
links_bb = ['https://www.bobbibrown.es/products/2330/makeup/eyes/sombra-de-ojos', \
           'https://www.bobbibrown.es/products/2339/makeup/lips']

In [150]:
bobbi_brown = Parallel(n_jobs=2, verbose=True)(delayed(extraer2)(e) for e in links_bb)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/4154559620.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_48607/4154559620.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   28.7s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   28.7s finished


In [151]:
sombras_bb = bobbi_brown[0]
labios_bb = bobbi_brown[1]

In [164]:
#ASIGNAMOS LAS PALETAS PARA CADA PRODUCTO
labios_bb['PRODUCTO'] = 'PINTALABIOS'
labios_bb.drop(labios_bb.index[[19]], inplace = True)
paletas = ['TODAS', 'PRIMAVERA / VERANO', 'TODAS', 'TODAS', 'PRIMAVERA', 'TODAS', 'TODAS','TODAS','TODAS',\
          'PRIMAVERA / OTOÑO / INVIERNO', 'TODAS', 'TODAS', 'TODAS', 'TODAS', 'PRIMAVERA / VERANO', 'TODAS',\
          'TODAS', 'PRIMAVERA / OTOÑO / INVIERNO', 'TODAS', 'PRIMAVERA', 'TODAS', 'OTOÑO / INVIERNO', \
          'TODAS', 'OTOÑO / INVIERNO', 'OTOÑO / INVIERNO']
labios_bb['PALETAS'] = paletas

In [154]:
#ASIGNAMOS LAS PALETAS PARA CADA PRODUCTO
sombras_bb.drop(sombras_bb.index[[0,1,4,5,14,15,16]], inplace = True)
paletas = ['PRIMAVERA / OTOÑO / INVIERNO', 'PRIMAVERA / OTOÑO / INVIERNO', 'PRIMAVERA / OTOÑO / VERANO', \
          'PRIMAVERA / OTOÑO / VERANO', 'PRIMAVERA / OTOÑO', 'TODAS', 'TODAS', 'PRIMAVERA / OTOÑO / VERANO', \
           'PRIMAVERA', 'PRIMAVERA / OTOÑO / INVIERNO', 'PRIMAVERA']
sombras_bb['PALETAS'] = paletas

# 8 - EXTRACCIÓN DE DATOS DE SOMBRAS, BLUSH Y PINTALABIOS DE NYX

In [ ]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

In [ ]:
URL = 'https://www.nyxcosmetics.es/paletas/sombras?start=0&sz=18#view-mode=grid'

In [ ]:
driver.get(URL)